In [1]:
import os

In [2]:
%pwd

'/Users/kshitij/MLFlow/research'

In [3]:
os.chdir("../")

In [4]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/vijayvergiya.kshitij98/MLFlow.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="vijayvergiya.kshitij98"
os.environ["MLFLOW_TRACKING_PASSWORD"]="49d9e27c0f040dd06a59c1c75fb1b4171503b5ed"

In [21]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [22]:
cd src

/Users/kshitij/MLFlow/src


In [23]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories, save_json

In [24]:
os.chdir("../")

In [16]:
%pwd

'/Users/kshitij/MLFlow'

In [25]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/vijayvergiya.kshitij98/MLFlow.mlflow",
           
        )

        return model_evaluation_config

In [26]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [27]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    
    def eval_metrics(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    


    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]


        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme


        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)

            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)
            
            # Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)


            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model, "model")



In [28]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2024-01-14 03:59:53,380: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-01-14 03:59:53,392: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-14 03:59:53,406: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-01-14 03:59:53,407: INFO: common: created directory at: artifacts]
[2024-01-14 03:59:53,409: INFO: common: created directory at: artifacts/model_evaluation]
[2024-01-14 03:59:53,917: INFO: common: json file saved at: artifacts/model_evaluation/metrics.json]


RestException: INTERNAL_ERROR: Response: {'error': 'unsupported endpoint, please contact support@dagshub.com'}

In [29]:
pip freeze | grep mlflow

Keyring is skipped due to an exception: 'keyring.backends'
mlflow==1.30.1
Note: you may need to restart the kernel to use updated packages.


In [30]:
pip install mlflow==2.2.2

Keyring is skipped due to an exception: 'keyring.backends'
ERROR: Could not find a version that satisfies the requirement mlflow==2.2.2 (from versions: 0.0.1, 0.1.0, 0.2.0, 0.2.1, 0.3.0, 0.4.0, 0.4.1, 0.4.2, 0.5.0, 0.5.1, 0.5.2, 0.6.0, 0.7.0, 0.8.0, 0.8.1, 0.8.2, 0.9.0, 0.9.0.1, 0.9.1, 1.0.0, 1.1.0, 1.1.1.dev0, 1.2.0, 1.3.0, 1.4.0, 1.5.0, 1.6.0, 1.7.0, 1.7.1, 1.7.2, 1.8.0, 1.9.0, 1.9.1, 1.10.0, 1.11.0, 1.12.0, 1.12.1, 1.13, 1.13.1, 1.14.0, 1.14.1, 1.15.0, 1.16.0, 1.17.0, 1.18.0, 1.19.0, 1.20.0, 1.20.1, 1.20.2, 1.21.0, 1.22.0, 1.23.0, 1.23.1, 1.24.0, 1.25.0, 1.25.1, 1.26.0, 1.26.1, 1.27.0, 1.28.0, 1.29.0, 1.30.0, 1.30.1)
ERROR: No matching distribution found for mlflow==2.2.2
You should consider upgrading via the '/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [33]:
pip freeze | grep mlflow

Keyring is skipped due to an exception: 'keyring.backends'
mlflow==1.30.1
Note: you may need to restart the kernel to use updated packages.
